# Problema 1

Tres refinerías con capacidades diarias de 6, 5 y 8 millones de galones, respectivamente, abastecen a tres áreas de distribución con demandas diarias de 4, 8 y 7 millones de galones, respectivamente. La gasolina se transporta a las tres áreas de distribución a través de una red de oleoductos. El costo de transporte es de \$0.10 por 1000 galones por kilómetro de oleoducto. En la tabla 1 se presenta la distancia en kilómetros entre las refinerías y las áreas de distribución. La refinería 1 no está conectada al área de distribución 3.

|             | Área 1 | Área 2 | Área 3 |
| ----------- | ------ | ------ | ------ |
| Refinería 1 | 120    | 180    | –      |
| Refinería 2 | 100    | 150    | 80     |
| Refinería 3 | 200    | 250    | 120    |

## Inciso a

Formular el modelo de transporte asociado.

In [ ]:
using Pkg

using JuMP
using HiGHS 

# Creamos el modelo
model = Model(HiGHS.Optimizer)

# Datos del problema
supply = [6, 5, 8]            # Refinerías
demand = [4, 8, 7]            # Áreas de distribución
costos = [
    12000  18000   Inf;
    30000  10000  8000;
    20000  25000 12000
]


3×3 Matrix{Float64}:
 12000.0  18000.0     Inf
 30000.0  10000.0   8000.0
 20000.0  25000.0  12000.0

## Inciso b

Usando JuMP o Pupl, determine el programa de envíos óptimo en la red de distribución.

In [3]:
# Variables de decisión: x[i,j] = cantidad enviada de refinería i a área j
@variable(model, x[1:3, 1:3] >= 0)

# Restricciones de oferta
for i in 1:3
    @constraint(model, sum(x[i,j] for j in 1:3) == supply[i])
end

# Restricciones de demanda
for j in 1:3
    @constraint(model, sum(x[i,j] for i in 1:3) == demand[j])
end

# Función objetivo: minimizar el costo total de transporte
@objective(model, Min, sum(costos[i,j] * x[i,j] for i in 1:3, j in 1:3 if isfinite(costos[i,j])))

# Resolver el modelo
optimize!(model)

# Mostrar resultados
println("Plan óptimo de envíos:")
for i in 1:3
    for j in 1:3
        if isfinite(costos[i,j])
            println("x[$i,$j] = ", value(x[i,j]))
        end
    end
end

println("Costo total mínimo: \$", objective_value(model))

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 6 rows; 9 cols; 18 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [8e+03, 3e+04]
  Bound  [0e+00, 0e+00]
  RHS    [4e+00, 8e+00]
Presolving model
6 rows, 9 cols, 18 nonzeros  0s
Dependent equations search running on 6 equations with time limit of 1000.00s
Dependent equations search removed 1 rows and 3 nonzeros in 0.00s (limit = 1000.00s)
5 rows, 9 cols, 15 nonzeros  0s
Presolve : Reductions: rows 5(-1); columns 9(-0); elements 15(-3)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 5(31) 0s
          5     2.1700000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 5
Objective value     :  2.1700000000e+05
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00
Pl

## Inciso c

Suponga ahora que la demanda diaria en el área 3 disminuye a 4 millones de galones. La producción excedente en las refinerías 1 y 2 se envía a otras áreas de distribución por medio de camiones. El costo de transporte por 100 galones es de \$1.50 desde la refinería 1 y de \$2.00 desde la refinería 2. La refinería 3 puede enviar su producción excedente a otros procesos químicos dentro de la planta.
Formule y resuelva de nuevo el programa óptimo de envíos.

In [4]:
using JuMP
using HiGHS

# Nuevo modelo
model_c = Model(HiGHS.Optimizer)

# Oferta
supply = [6, 5, 8]  # Total: 19

# Nueva demanda con destino ficticio
demand = [4, 8, 4, 3]  # Total: 19

# Nueva matriz de costos incluyendo el destino ficticio
costos = [
    12000  18000   Inf   1500;
    30000  10000  8000   2000;
    20000  25000 12000   Inf
]

# Variables: ahora 3x4
@variable(model_c, x[1:3, 1:4] >= 0)

# Restricciones de oferta (suministro)
for i in 1:3
    @constraint(model_c, sum(x[i,j] for j in 1:4) == supply[i])
end

# Restricciones de demanda (demanda + ficticio)
for j in 1:4
    @constraint(model_c, sum(x[i,j] for i in 1:3) == demand[j])
end

# Función objetivo con condición para ignorar Infinito
@objective(model_c, Min, sum(costos[i,j] * x[i,j] for i in 1:3, j in 1:4 if isfinite(costos[i,j])))

# Resolver modelo
optimize!(model_c)

# Mostrar resultados
println("Programa óptimo de envíos con destino ficticio:")
for i in 1:3
    for j in 1:4
        if isfinite(costos[i,j])
            println("x[$i,$j] = ", value(x[i,j]))
        end
    end
end

println("Costo total mínimo (con destino ficticio): \$", objective_value(model_c))


Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 7 rows; 12 cols; 24 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [2e+03, 3e+04]
  Bound  [0e+00, 0e+00]
  RHS    [3e+00, 8e+00]
Presolving model
7 rows, 12 cols, 24 nonzeros  0s
Dependent equations search running on 7 equations with time limit of 1000.00s
Dependent equations search removed 1 rows and 4 nonzeros in 0.00s (limit = 1000.00s)
6 rows, 12 cols, 20 nonzeros  0s
Presolve : Reductions: rows 6(-1); columns 12(-0); elements 20(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 6(30) 0s
          7     1.8900000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 7
Objective value     :  1.8900000000e+05
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.0